In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot
from numpy import cos, sin, exp, min, max
from numpy.linalg import eigh
from ipywidgets import interact
import matplotlib
from matplotlib import rcParams
pi = np.pi

'''
General program to track Weyl nodes in reciprocal space in the generalized anizotropic BHZ model
you can set u_min and u_max for a fixed C to track the Weyl nodes in that regime. You can also modify vx (the anisotropy parameter,
i.e. the prefactor of hopping in the x/kx direction). kres is the resolution of the BZ, u_res is the number of datapoints
that it collects between u_min and u_max. C is the parameter of C1, i.e. the ty*sy coupling term
'''

rcParams['figure.figsize'] = 6, 5
rcParams.update({'font.size': 10})
vx=0.8

#U_4 is the matrix we use to bring H_BHZ into the chiral basis
U_4 = 1/np.sqrt(2.)*np.array([[1, 0, 0, -1j],[0, 1, -1j, 0],[0, 1, 1j, 0],[1, 0, 0, 1j]])
   
matplotlib.rcParams['figure.dpi'] = 100
def plotter(CC = 1, kres = 10, u_res = 10, break_inv_1 = 0, break_inv_2=0, break_time_rev=0, u_min = -2.5, u_max = 4, vx=0.8): 
    uvals = np.linspace(u_min, u_max, u_res)
    emin, emax = [], []
    # Resolution of half of Brillouin zone: kres
    Q = np.zeros([2*kres, 2*kres])
    #putting together the Hamiltonian - the first two indeces are kx and ky to avoid an extra two for loops
    for u in uvals:
        HH = np.zeros((2*kres, 2*kres, 4, 4), dtype=complex)
        kvals = np.linspace(-pi, pi, 2*kres, endpoint=False)
        ky, kx = np.meshgrid(kvals, kvals)
        HH[:, :, 0, 0] = u+vx*cos(kx)+cos(ky)
        HH[:, :, 0, 1] = vx*sin(kx)-1.j*sin(ky)
        HH[:, :, 1, 0] =  HH[:, :, 0, 1].conj() 
        HH[:, :, 1, 1] = -HH[:, :, 0, 0]
        HH[:, :, 2, 2] = u+vx*cos(kx)+cos(ky)
        HH[:, :, 2, 3] = -vx*sin(kx)-1.j*sin(ky)
        HH[:, :, 3, 2] =  HH[:, :, 2, 3].conj() 
        HH[:, :, 3, 3] = -HH[:, :, 2, 2]
        HH[:, :, 0, 3] = -CC
        HH[:, :, 1, 2] = -HH[:, :, 0, 3] 
        HH[:, :, 2, 1] = HH[:, :, 1, 2].conj()
        HH[:, :, 3, 0] = HH[:, :, 0, 3].conj()
        
        #inv symm breaking: sig_x tau_x sin(k)
        
        HH[:, :, 0, 3] += break_inv_1*(sin(kx)+sin(ky))
        HH[:, :, 1, 2] += break_inv_1*(sin(kx)+sin(ky))
        HH[:, :, 2, 1] += break_inv_1*(sin(kx)+sin(ky))
        HH[:, :, 3, 0] += break_inv_1*(sin(kx)+sin(ky))
        
        #inv symm breaking: sig_0 tau_z sin(k)
        
        HH[:, :, 0, 0] += break_inv_2* (sin(kx)+sin(ky))
        HH[:, :, 1, 1] += break_inv_2* (sin(kx)+sin(ky))
        HH[:, :, 2, 2] -= break_inv_2* (sin(kx)+sin(ky))
        HH[:, :, 3, 2] -= break_inv_2* (sin(kx)+sin(ky))
        
        if(break_time_rev == 1): #tau_0 sig_z sin(k)
            HH[:, :, 0, 0] += sin(kx)+sin(ky)
            HH[:, :, 1, 1] -= sin(kx)+sin(ky)
            HH[:, :, 2, 2] += sin(kx)+sin(ky)
            HH[:, :, 3, 2] -= sin(kx)+sin(ky)
        
                
        #HH_antidiag is the chiral form
        HH_antidiag = (np.matmul(np.matmul(U_4,HH), (U_4.conj().T)))
        dett = HH_antidiag[:, :, 0, 2]*HH_antidiag[:, :, 1, 3] \
               - HH_antidiag[:, :, 0, 3]*HH_antidiag[:, :, 1, 2]
        #print(HH_antidiag[0, 0])
        #print(HH_antidiag[1, 2])
        #print(HH_antidiag[4, 6]) #check if block off-diagonalization worked
        #print(HH_antidiag[17, 12])
        
        dettx  = np.roll(dett, 1, axis=0)
        detty  = np.roll(dett, 1, axis=1)
        dettxy = np.roll(np.roll(dett, 1, axis=1), 1, axis=0)
    
        Q += (np.angle(dett/dettx) + np.angle(dettx/dettxy)  \
            + np.angle(dettxy/detty) + np.angle(detty/dett)) /2/pi
        
        
        
    #we are only interested in the sign of the total summed chiralities, as otherwise if a node moves slowly in one
    #region, then it will be coloured differently if the same node moves faster in another region.
    sign = np.zeros([2*kres, 2*kres])
    
    for i in range(0, 2*kres):
        for j in range(0, 2*kres):
            if(abs(Q[i, j])<0.1): sign[i, j] = 0
            else: sign[i, j] = np.sign(Q[i, j])

    plt.pcolormesh(np.linspace(-1, 1, 2*kres, endpoint=False),np.linspace(-1, 1, 2*kres, endpoint=False), sign.T)
    x_label = r"$k_x$" +" in units of " "$\pi$"
    y_label = r"$k_y$" +" in units of " "$\pi$"
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title("Tracking Weyl nodes in the Brillouin zone")
    #plt.title(str("bi1: " + str(break_inv_1) + " bi2: " + str(break_inv_2) + " CC is" + str(CC)))
    plt.colorbar()
    plt.savefig("cover.pdf")
    plt.show()
interact(plotter, CC=(0, 4, 0.01), kres = (10, 310, 20), u_res = (40, 8000, 80),\
         break_inv_1 = (0, 1, 0.1), break_inv_2 = (0, 1, 0.1), break_time_rev = (0, 1, 1), u_min = (-3, 3, 0.005), u_max = (-3, 3, 0.005), vx=(0.1, 1, 0.05))

interactive(children=(FloatSlider(value=1.0, description='CC', max=4.0, step=0.01), IntSlider(value=10, descri…

<function __main__.plotter(CC=1, kres=10, u_res=10, break_inv_1=0, break_inv_2=0, break_time_rev=0, u_min=-2.5, u_max=4, vx=0.8)>

<Figure size 600x500 with 0 Axes>

In [5]:
'''
same code as above, but instead of keeping C fixed and varying u in a regime,
here we: keep u fixed and vary C from C_min to C_max
'''

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot
from numpy import cos, sin, exp, min, max
from numpy.linalg import eigh
from ipywidgets import interact
import matplotlib
from matplotlib import rcParams
pi = np.pi

rcParams['figure.figsize'] = 6, 5
rcParams.update({'font.size': 10})

U_4 = 1/np.sqrt(2.)*np.array([[1, 0, 0, -1j],[0, 1, -1j, 0],[0, 1, 1j, 0],[1, 0, 0, 1j]])
   
matplotlib.rcParams['figure.dpi'] = 100
def plotter(u = -0.5,  kres = 10, CC_res = 10, break_inv_1 = 0, break_inv_2=0, break_time_rev=0, CC_min = -2.5, CC_max = 4): 
    
    
    CCvals = np.linspace(CC_min, CC_max, CC_res)
    emin, emax = [], []
    # Resolution of half of Brillouin zone: kres
    Q = np.zeros([2*kres, 2*kres])
    for CC in CCvals:
        HH = np.zeros((2*kres, 2*kres, 4, 4), dtype=complex)
        kvals = np.linspace(-pi, pi, 2*kres, endpoint=False)
        ky, kx = np.meshgrid(kvals, kvals)
        HH[:, :, 0, 0] = u+cos(kx)+cos(ky)
        HH[:, :, 0, 1] = sin(kx)-1.j*sin(ky)
        HH[:, :, 1, 0] =  HH[:, :, 0, 1].conj() 
        HH[:, :, 1, 1] = -HH[:, :, 0, 0]
        HH[:, :, 2, 2] = u+cos(kx)+cos(ky)
        HH[:, :, 2, 3] = -sin(kx)-1.j*sin(ky)
        HH[:, :, 3, 2] =  HH[:, :, 2, 3].conj() 
        HH[:, :, 3, 3] = -HH[:, :, 2, 2]
        HH[:, :, 0, 3] = -CC
        HH[:, :, 1, 2] = -HH[:, :, 0, 3] 
        HH[:, :, 2, 1] = HH[:, :, 1, 2].conj()
        HH[:, :, 3, 0] = HH[:, :, 0, 3].conj()
        
        #inv symm breaking: sig_x tau_x sin(k)
        
        HH[:, :, 0, 3] += break_inv_1*(sin(kx)+sin(ky))
        HH[:, :, 1, 2] += break_inv_1*(sin(kx)+sin(ky))
        HH[:, :, 2, 1] += break_inv_1*(sin(kx)+sin(ky))
        HH[:, :, 3, 0] += break_inv_1*(sin(kx)+sin(ky))
        
        #inv symm breaking: sig_0 tau_z sin(k)
        
        HH[:, :, 0, 0] += break_inv_2* (sin(kx)+sin(ky))
        HH[:, :, 1, 1] += break_inv_2* (sin(kx)+sin(ky))
        HH[:, :, 2, 2] -= break_inv_2* (sin(kx)+sin(ky))
        HH[:, :, 3, 2] -= break_inv_2* (sin(kx)+sin(ky))
        
        if(break_time_rev == 1): #tau_0 sig_z sin(k)
            HH[:, :, 0, 0] += sin(kx)+sin(ky)
            HH[:, :, 1, 1] -= sin(kx)+sin(ky)
            HH[:, :, 2, 2] += sin(kx)+sin(ky)
            HH[:, :, 3, 2] -= sin(kx)+sin(ky)
        
                
       
        HH_antidiag = (np.matmul(np.matmul(U_4,HH), (U_4.conj().T)))
        dett = HH_antidiag[:, :, 0, 2]*HH_antidiag[:, :, 1, 3] \
               - HH_antidiag[:, :, 0, 3]*HH_antidiag[:, :, 1, 2]
        #print(HH_antidiag[0, 0])
        #print(HH_antidiag[1, 2])
        #print(HH_antidiag[4, 6]) #check if block off-diagonalization worked
        #print(HH_antidiag[17, 12])
        
        dettx  = np.roll(dett, 1, axis=0)
        detty  = np.roll(dett, 1, axis=1)
        dettxy = np.roll(np.roll(dett, 1, axis=1), 1, axis=0)
    
        Q += (np.angle(dett/dettx) + np.angle(dettx/dettxy)  \
            + np.angle(dettxy/detty) + np.angle(detty/dett)) /2/pi
        
        
        
        #ee, vv = eigh(HH)
        #eee = ee.flatten()
        #emin.append(min(eee[eee>0]))
        #emax.append(max(eee[eee<0]))
    #plot(uvals, emin)
    #plot(uvals, emax)
    #plt.show()

    sign = np.zeros([2*kres, 2*kres])
    
    for i in range(0, 2*kres):
        for j in range(0, 2*kres):
            if(abs(Q[i, j])<0.1): sign[i, j] = 0
            else: sign[i, j] = np.sign(Q[i, j])

    plt.pcolormesh(np.linspace(-1, 1, 2*kres, endpoint=False),np.linspace(-1, 1, 2*kres, endpoint=False), sign.T)
    x_label = r"$k_x$" +" in units of " "$\pi$"
    y_label = r"$k_y$" +" in units of " "$\pi$"
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title("Tracking Weyl nodes in the BZ")
    #plt.title(str("bi1: " + str(break_inv_1) + " bi2: " + str(break_inv_2) + " CC is" + str(CC)))
    #plt.colorbar()
    plt.savefig("nodertackerrr3v.png")
    plt.show()
interact(plotter, u=(-2.5, 2.5, 0.01), kres = (10, 100, 10), CC_res = (40, 600, 80),\
         break_inv_1 = (0, 1, 0.1), break_inv_2 = (0, 1, 0.1), break_time_rev = (0, 1, 1), CC_min = (0, 2, 0.05), CC_max = (0, 2, 0.05))

interactive(children=(FloatSlider(value=-0.5, description='u', max=2.5, min=-2.5, step=0.01), IntSlider(value=…

<function __main__.plotter(u=-0.5, kres=10, CC_res=10, break_inv_1=0, break_inv_2=0, break_time_rev=0, CC_min=-2.5, CC_max=4)>